# Módulo 08 - Estruturas de Dados

## GeoSeries vs GeoDataframe

A `GeoSeries` é um vetor onde cada entrada no vetor é um conjunto de formas correspondentes a uma observação, por exemplo, um único polígono.

Já o `GeoDataframe` é uma estrutura de dados que contém um arquivo `GeoSeries`.

In [ ]:
!pip install click==8.1.3
!pip install geopandas
import geopandas as gpd

In [ ]:
from shapely.geometry import Polygon, LineString, Point

s = gpd.GeoSeries(
    [
        Polygon([(0, 0), (1,1), (0,1)]),
        Polygon([(10, 0), (10,5), (0,0)]),
        Polygon([(0, 0), (2,2), (2,0)]),
        LineString([(0, 0), (1,1), (0,1)]),
        Point(0,1)
    ]

)
s

In [ ]:
type(s)

## Diferença entre Dataframe e GeoDataframe



In [ ]:
import geopandas as gpd
import os
from osgeo import gdal
gdal.SetConfigOption('SHAPE_RESTORE_SHX', 'YES')
in_vector = 'C:\Shapefiles\dados_curso_basico\Regionalizacao_BA\gisdb_dados_sei_REG_MUN_A_100K_2022_GCS_SIR_SEI.shp'
gdf = gpd.read_file(in_vector)

In [ ]:
# Definir o caminho da pasta
pasta = r'C:\Shapefiles\dados_curso_basico\Regionalizacao_BA\gisdb_dados_sei_REG_MUN_A_100K_2022_GCS_SIR_SEI.shp'
#pasta = "/content/DPA_A_100K_2022_GCS_SIR_SEI.shp"

# Juntar o nome do arquivo com a pasta
#pasta_arquivo = os.path.join(pasta, 'BR_Municipios_2021.shp')
print(pasta)
#print(pasta_arquivo)

In [ ]:
dados = gpd.read_file(pasta)

In [ ]:
type(dados)

In [ ]:
dados.head(10)

In [ ]:
dados.tail()

In [ ]:
# Verificar o nome das colunas
dados.columns.values

array(['codmun', 'nommun', 'codregmeso', 'nomregmeso', 'codregmicr',
       'nomregmicr', 'codreginte', 'nomreginte', 'codregimed',
       'nomregimed', 'indregpoli', 'indregsemi', 'codti', 'nomti',
       'codregecon', 'nomregecon', 'cod_regadm', 'nomregadm',
       'codeixodes', 'nomeixodes', 'codnte', 'nomnte', 'codnrs', 'nomnrs',
       'codregsau', 'nomregsau', 'codzonatur', 'nomzonatur', 'st_area_sh',
       'st_length_', 'geometry'], dtype=object)

In [ ]:
dados = dados[['codmun','nommun','codregmeso', 'nomregmeso', 'codti', 'nomti', 'st_area_sh', 'st_length_', 'geometry']]
dados.head()

In [ ]:
dados.shape

In [ ]:
# Verificando o número de linhas e colunas
print(f'O número de linhas é: {dados.shape[0]}')
print(f'O número de colunas é: {dados.shape[1]}')

In [ ]:
dados.plot()

In [ ]:
for indice, linha in dados[0:10].iterrows():
    area_polig = linha['geometry'].area
    print(f"O polígono de índice {indice} possui uma área de {area_polig:.2f} m²")
    

In [ ]:
# Criar uma coluna área em km2
dados['Area'] = dados.area
dados.head()

In [ ]:
# Apagando a coluna area por estar fora do padrão de nome de atributo
dados = dados.drop(columns=['Area'])
dados.head()

In [ ]:
# Criar uma coluna área em km2
dados['area_km2'] = dados.area
dados.head()

In [ ]:
# Transformando o atributo st_area_sh em km2
dados['st_area_sh'] = dados['area_km2'] * 100
dados.head()

In [ ]:
# área máxima de st_area_sh
round(dados['st_area_sh'].max(), 2)

In [ ]:
# área mínima de st_area_sh
dados['st_area_sh'].min()

In [ ]:
# área média de st_area_sh
dados['st_area_sh'].mean()

In [ ]:
# Soma de todas as áreas de st_area_sh 
dados['st_area_sh'].sum() / dados['st_area_sh'].shape[0]

# Selecionando um Município e Salvando

In [ ]:
select = dados.loc[dados['nommun'] == 'Senhor do Bonfim']

In [ ]:
select.head()

In [ ]:
select.plot()

In [ ]:
pasta_saida = r"C:\Shapefiles\dados_curso_basico\Reg_BA\Municipios"

# Criar pasta
os.mkdir(pasta_saida)

In [ ]:
arquivo_saida = os.path.join(pasta_saida, "BA_Senhor_do_Bonfim.shp")
print(arquivo_saida)

In [ ]:
# Salvando o arquivo
select.to_file(arquivo_saida)

# Agrupando GeoDataframe

In [ ]:
dados.head()

In [ ]:
# DPA da Bahia com informações de TI e agrupar os municípios por TI
dados['nomti'].unique()

In [ ]:
agrupar = dados.groupby('nomti')
agrupar

In [ ]:
agrupar.groups.keys()

In [ ]:
for chave, grupo in agrupar:
    print(f'Nome do Território Identidade: {chave}')
    print(f'Número de Municípios: {len(grupo)}')

In [ ]:
arquivo_saida = r"C:\Shapefiles\dados_curso_basico\Reg_BA\Municipios"

resultado = os.path.join(arquivo_saida, "Resultados")

if not os.path.exists(resultado):
    print("Criando a pasta.")
    os.makedirs(resultado)
else:
    print("A pasta ja existe")

In [ ]:
agrupar.head()

In [ ]:
for chave, grupo in agrupar:
    nome_saida = f'{chave}.shp'
    print('Salvando o arquivo', os.path.basename(nome_saida))
    caminho_saida = os.path.join(resultado, nome_saida)
    grupo.to_file(caminho_saida)

# Trabalhando com Coordenadas

Para saber o código do EPSG da região de interesse existem diversos sites, mas eu costumo utilizar esse aqui: 
https://spatialreference.org/

In [ ]:
import geopandas as gpd

In [ ]:
arquivo = r"C:\Shapefiles\dados_curso_basico\Reg_BA\Municipios\Resultados\Bacia do Jacuípe.shp"

In [ ]:
dados_bacia_jacuipe = gpd.read_file(arquivo)
dados_bacia_jacuipe.head()

In [ ]:
# Verificar o sistema de coordenadas
dados_bacia_jacuipe.crs

In [ ]:
# Criar uma cópia do arquivo
dados_bacia_jacuipe_epsg_4674 = dados_bacia_jacuipe.copy()

# Reprojetar para a coordenada da região
dados_bacia_jacuipe = dados_bacia_jacuipe.to_crs(epsg=31982)

In [ ]:
dados_bacia_jacuipe.crs

In [ ]:
dados_bacia_jacuipe.head()

In [ ]:
dados_bacia_jacuipe['area_km2'] = dados_bacia_jacuipe.area
dados_bacia_jacuipe.head()

In [ ]:
dados_bacia_jacuipe_epsg_4674.head()

In [ ]:
import matplotlib.pyplot as plt

# Fazendo um plot do lado do outro

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10,10))

dados_bacia_jacuipe_epsg_4674.plot(ax=ax1, facecolor='gray')
dados_bacia_jacuipe.plot(ax=ax2, facecolor='gray')

# Definir a proporção como 1
ax1.set_aspect(aspect=1)
ax2.set_aspect(aspect=1)


In [ ]:
# Criando o path do arquivo de saída EPSG 31982 e EPSG 4674
arquivo_saida_epsg31982 = r'C:\Shapefiles\dados_curso_basico\Reg_BA\Municipios\Resultados\Bacia_Jacuipe_epsg31982.shp'
arquivo_saida_epsg4674 = r'C:\Shapefiles\dados_curso_basico\Reg_BA\Municipios\Resultados\Bacia_Jacuipe_epsg4674.shp'
arquivo_saida_epsg31982

In [ ]:
arquivo_saida_epsg4674

In [ ]:
# Salvar o arquivo
dados_bacia_jacuipe.to_file(arquivo_saida_epsg31982)
dados_bacia_jacuipe.to_file(arquivo_saida_epsg4674)

# Fim do módulo 8